# Sample Notebook for Zero-Shot Inference with CheXzero
This notebook walks through how to use CheXzero to perform zero-shot inference on a chest x-ray image dataset.

## Import Libraries

In [1]:
import os
import numpy as np
import pandas as pd
from pathlib import Path
from typing import List, Tuple, Optional

import sys
sys.path.append(r'C:\Users\Vishi\VSC Codes\VIsLM_seminar\VLP-Seminar\cheXzeroCode')

from eval import evaluate, bootstrap
from zero_shot import make, make_true_labels, run_softmax_eval

%load_ext autoreload
%autoreload 2

## Directories and Constants

In [2]:
cheXpert_data_dir =  r"D:\CheXpert"

In [3]:
## Define Zero Shot Labels and Templates

# ----- DIRECTORIES ------ #
cxr_filepath: str = r'C:\Users\Vishi\VSC Codes\VIsLM_seminar\VLP-Seminar\cheXzeroCode\cxr.h5' # filepath of chest x-ray images (.h5)
cxr_true_labels_path: Optional[str] = 'D:/CheXpert/groundtruth.csv' # (optional for evaluation) if labels are provided, provide path
model_dir: str = 'C:/Users/Vishi/VSC Codes/VIsLM_seminar/VLP-Seminar/data/checkpoints/chexZero checkpoints' # where pretrained models are saved (.pt) 
predictions_dir: Path = Path('../predictions') # where to save predictions
cache_dir: str = predictions_dir / "cached" # where to cache ensembled predictions

context_length: int = 77

# ------- LABELS ------  #
# Define labels to query each image | will return a prediction for each label
cxr_labels: List[str] = ['Atelectasis','Cardiomegaly', 
                                      'Consolidation', 'Edema', 'Enlarged Cardiomediastinum', 'Fracture', 'Lung Lesion',
                                      'Lung Opacity', 'No Finding','Pleural Effusion', 'Pleural Other', 'Pneumonia', 
                                      'Pneumothorax', 'Support Devices']

# ---- TEMPLATES ----- # 
# Define set of templates | see Figure 1 for more details  

pathology = "Cardiomegaly"                      
cxr_pair_template: Tuple[str] = (f"{pathology}", f"no {pathology}")

# ----- MODEL PATHS ------ #
# If using ensemble, collect all model paths
model_paths = []
# print(os.walk(model_dir).__getattribute__)
for subdir, dirs, files in os.walk(model_dir):
    for file in files:
        full_dir = os.path.join(subdir, file)
        model_paths.append(full_dir)
        
print(model_paths)

['C:/Users/Vishi/VSC Codes/VIsLM_seminar/VLP-Seminar/data/checkpoints/chexZero checkpoints\\best_128_0.0002_original_15000_0.859.pt', 'C:/Users/Vishi/VSC Codes/VIsLM_seminar/VLP-Seminar/data/checkpoints/chexZero checkpoints\\best_128_0.0002_original_8000_0.857.pt', 'C:/Users/Vishi/VSC Codes/VIsLM_seminar/VLP-Seminar/data/checkpoints/chexZero checkpoints\\best_128_5e-05_original_22000_0.855.pt', 'C:/Users/Vishi/VSC Codes/VIsLM_seminar/VLP-Seminar/data/checkpoints/chexZero checkpoints\\best_64_0.0001_original_16000_0.861.pt', 'C:/Users/Vishi/VSC Codes/VIsLM_seminar/VLP-Seminar/data/checkpoints/chexZero checkpoints\\best_64_0.0001_original_17000_0.863.pt', 'C:/Users/Vishi/VSC Codes/VIsLM_seminar/VLP-Seminar/data/checkpoints/chexZero checkpoints\\best_64_0.0001_original_35000_0.864.pt', 'C:/Users/Vishi/VSC Codes/VIsLM_seminar/VLP-Seminar/data/checkpoints/chexZero checkpoints\\best_64_0.0002_original_23000_0.854.pt', 'C:/Users/Vishi/VSC Codes/VIsLM_seminar/VLP-Seminar/data/checkpoints/chexZ

In [4]:
# # #save all frontal images to a new folder frontal
# import os
# import shutil

# # Define paths
# base_folder = r"D:\CheXpert\valid"  # Replace with the actual path
# output_folder = r"valid_frontal"  # Replace with the actual path

# # Create the output folder if it doesn't exist
# os.makedirs(output_folder, exist_ok=True)

# # Copy all frontal images to the output folder
# counter = 0
# for root, dirs, files in os.walk(base_folder):
#     for file in files:
#         if file.endswith("frontal.jpg"):

#             # get grandparent folder name
#             parent_folder = os.path.basename(os.path.dirname(root))
#             print(parent_folder)
#             newName = f"{parent_folder}_frontal_{counter}.jpg"
#             img_path = os.path.join(output_folder, newName)
#             shutil.copy(os.path.join(root, file), img_path)
#             counter += 1

In [5]:
# #convert all images in C:\Users\Vishi\VSC Codes\VIsLM_seminar\VLP-Seminar\notebooks\valid_frontal from .jpg to .png
# import os
# from PIL import Image

# # Define paths
# base_folder = r"C:\Users\Vishi\VSC Codes\VIsLM_seminar\VLP-Seminar\notebooks\valid_frontal"  # Replace with the actual path

# # Convert all images to .png
# for root, dirs, files in os.walk(base_folder):
#     for file in files:
#         if file.endswith(".jpg"):
#             img_path = os.path.join(root, file)
#             img = Image.open(img_path)
#             img.save(img_path.replace(".jpg", ".png"))
#             os.remove(img_path)


In [6]:
# #get list of image paths in C:\Users\Vishi\VSC Codes\VIsLM_seminar\VLP-Seminar\notebooks\valid_frontal
# import os

# # Define paths
# base_folder = r"C:\Users\Vishi\VSC Codes\VIsLM_seminar\VLP-Seminar\notebooks\valid_frontal"  # Replace with the actual path

# # Get all image paths
# image_paths = []
# for root, dirs, files in os.walk(base_folder):
#     for file in files:
#         if file.endswith(".png"):
#             image_paths.append(os.path.join(root, file))

# image_paths

In [7]:
# from preprocess_padchest import img_to_h5

# proper_paths = img_to_h5(image_paths, r"valid.h5")

## Run Inference

In [9]:
# cxr_filepath

# # If using HDF5 file directly
# with h5py.File(cxr_filepath, 'r') as f:
#     dataset = f['cxr']
#     dataset = torch.tensor(dataset)

# dataset

In [11]:
import subprocess
import numpy as np
import os
import sys
import pandas as pd
from PIL import Image
import h5py
import matplotlib.pyplot as plt
from typing import List, Tuple

import torch
from torch.utils import data
from tqdm.notebook import tqdm
import torch.nn as nn
from torchvision.transforms import Compose, Normalize, Resize, InterpolationMode

import sklearn
from sklearn.metrics import confusion_matrix, accuracy_score, auc, roc_auc_score, roc_curve, classification_report
from sklearn.metrics import precision_recall_curve, f1_score
from sklearn.metrics import average_precision_score
from pathlib import Path


import clip
from model import CLIP
from eval import evaluate, plot_roc, accuracy, sigmoid, bootstrap, compute_cis

In [12]:
def run_softmax_eval(model, loader, eval_labels: list, pair_template: tuple, context_length: int = 77): 
    """
    Run softmax evaluation to obtain a single prediction from the model.
    """
     # get pos and neg phrases
    pos = pair_template[0]
    neg = pair_template[1]

    # get pos and neg predictions, (num_samples, num_classes)
    pos_pred = run_single_prediction(eval_labels, pos, model, loader, 
                                     softmax_eval=True, context_length=context_length) 
    neg_pred = run_single_prediction(eval_labels, neg, model, loader, 
                                     softmax_eval=True, context_length=context_length) 

    # compute probabilities with softmax
    sum_pred = np.exp(pos_pred) + np.exp(neg_pred)
    y_pred = np.exp(pos_pred) / sum_pred
    return y_pred
def run_single_prediction(cxr_labels, template, model, loader, softmax_eval=True, context_length=77): 
    """
    FUNCTION: run_single_prediction
    --------------------------------------
    This function will make probability predictions for a single template
    (i.e. "has {}"). 
    
    args: 
        * cxr_labels - list, labels for a specific zero-shot task. (i.e. ['Atelectasis',...])
        * template - string, template to input into model. 
        * model - PyTorch model, trained clip model
        * loader - PyTorch data loader, loads in cxr images
        * softmax_eval (optional) - Use +/- softmax method for evaluation 
        * context_length (optional) - int, max number of tokens of text inputted into the model.
        
    Returns list, predictions from the given template. 
    """
    cxr_phrase = [template]
    zeroshot_weights = zeroshot_classifier(cxr_labels, cxr_phrase, model, context_length=context_length)
    y_pred = predict(loader, model, zeroshot_weights, softmax_eval=softmax_eval)
    return y_pred

def zeroshot_classifier(classnames, templates, model, context_length=77):
    """
    FUNCTION: zeroshot_classifier
    -------------------------------------
    This function outputs the weights for each of the classes based on the 
    output of the trained clip model text transformer. 
    
    args: 
    * classnames - Python list of classes for a specific zero-shot task. (i.e. ['Atelectasis',...]).
    * templates - Python list of phrases that will be indpendently tested as input to the clip model.
    * model - Pytorch model, full trained clip model.
    * context_length (optional) - int, max number of tokens of text inputted into the model.
    
    Returns PyTorch Tensor, output of the text encoder given templates. 
    """
    with torch.no_grad():
        zeroshot_weights = []
        # compute embedding through model for each class
        for classname in tqdm(classnames):
            texts = [template.format(classname) for template in templates] # format with class
            texts = clip.tokenize(texts, context_length=context_length) # tokenize
            class_embeddings = model.encode_text(texts) # embed with text encoder
            
            # normalize class_embeddings
            class_embeddings /= class_embeddings.norm(dim=-1, keepdim=True)
            # average over templates 
            class_embedding = class_embeddings.mean(dim=0) 
            # norm over new averaged templates
            class_embedding /= class_embedding.norm() 
            zeroshot_weights.append(class_embedding)
        zeroshot_weights = torch.stack(zeroshot_weights, dim=1)
    return zeroshot_weights

def predict(loader, model, zeroshot_weights, softmax_eval=True, verbose=0): 
    """
    FUNCTION: predict
    ---------------------------------
    This function runs the cxr images through the model 
    and computes the cosine similarities between the images
    and the text embeddings. 
    
    args: 
        * loader -  PyTorch data loader, loads in cxr images
        * model - PyTorch model, trained clip model 
        * zeroshot_weights - PyTorch Tensor, outputs of text encoder for labels
        * softmax_eval (optional) - Use +/- softmax method for evaluation 
        * verbose (optional) - bool, If True, will print out intermediate tensor values for debugging.
        
    Returns numpy array, predictions on all test data samples. 
    """
    y_pred = []
    with torch.no_grad():
        for i, data in enumerate(tqdm(loader)):
            images = data['img']
            # predict
            image_features = model.encode_image(images) 
            image_features /= image_features.norm(dim=-1, keepdim=True) # (1, 768)

            # obtain logits
            logits = image_features @ zeroshot_weights # (1, num_classes)
            logits = np.squeeze(logits.numpy(), axis=0) # (num_classes,)
        
            if softmax_eval is False: 
                norm_logits = (logits - logits.mean()) / (logits.std())
                logits = sigmoid(norm_logits) 
            
            y_pred.append(logits)
            
            if verbose: 
                plt.imshow(images[0][0])
                plt.show()
                print('images: ', images)
                print('images size: ', images.size())
                
                print('image_features size: ', image_features.size())
                print('logits: ', logits)
                print('logits size: ', logits.size())
         
    y_pred = np.array(y_pred)
    return np.array(y_pred)


In [14]:
## Run the model on the data set using ensembled models
def ensemble_models(
    model_paths: List[str], 
    cxr_filepath: str, 
    cxr_labels: List[str], 
    cxr_pair_template: Tuple[str], 
    cache_dir: str = None, 
    save_name: str = None,
) -> Tuple[List[np.ndarray], np.ndarray]: 
    """
    Given a list of `model_paths`, ensemble model and return
    predictions. Caches predictions at `cache_dir` if location provided.

    Returns a list of each model's predictions and the averaged
    set of predictions.
    """

    predictions = []
    model_paths = sorted(model_paths) # ensure consistency of 
    for path in model_paths: # for each model
        model_name = Path(path).stem

        # load in model and `torch.DataLoader`
        model, loader = make(
            model_path=path, 
            cxr_filepath=cxr_filepath, 
        )
        # print(type(loader))  # Should print <class 'torch.utils.data.dataloader.DataLoader'>
        # print(type(loader.dataset)) 
        # print(len(loader))
        dataset = loader.dataset
        dataset = dataset.img_dset
        #print dataset attributes
        print(type(dataset))
        #get type of dataset content
        if cache_dir is not None:
            if save_name is not None: 
                cache_path = Path(cache_dir) / f"{save_name}_{model_name}.npy"
            else: 
                cache_path = Path(cache_dir) / f"{model_name}.npy"

        # if prediction already cached, don't recompute prediction
        if False:#cache_dir is not None and os.path.exists(cache_path): 
            print("Loading cached prediction for {}".format(model_name))
            y_pred = np.load(cache_path)
        else: # cached prediction not found, compute preds
            # print("Inferring model {}".format(path))
            # print("Model name: {}".format(model_name))
            # print(cxr_labels)
            # print(cxr_pair_template)
            y_pred = run_softmax_eval(model, loader, cxr_labels, cxr_pair_template)
            print(y_pred)
            if cache_dir is not None: 
                Path(cache_dir).mkdir(exist_ok=True, parents=True)
                np.save(file=cache_path, arr=y_pred)
        predictions.append(y_pred)
    
    # compute average predictions
    y_pred_avg = np.mean(predictions, axis=0)
    
    return predictions, y_pred_avg

predictions, y_pred_avg = ensemble_models(
    model_paths=model_paths, 
    cxr_filepath=cxr_filepath, 
    cxr_labels=cxr_labels, 
    cxr_pair_template=cxr_pair_template, 
    cache_dir=cache_dir,
)

<class 'torch.Tensor'>


  0%|          | 0/14 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/14 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

...

  0%|          | 0/14 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/14 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

[[0.49674878 0.49674878 0.49674878 ... 0.49674878 0.49674878 0.49674878]
 [0.48555338 0.48555338 0.48555338 ... 0.48555338 0.48555338 0.48555338]
 [0.49606162 0.49606162 0.49606162 ... 0.49606162 0.49606162 0.49606162]
 ...
 [0.4972743  0.4972743  0.4972743  ... 0.4972743  0.4972743  0.4972743 ]
 [0.4977789  0.4977789  0.4977789  ... 0.4977789  0.4977789  0.4977789 ]
 [0.4932244  0.4932244  0.4932244  ... 0.4932244  0.4932244  0.4932244 ]]


In [15]:
# save averaged preds
pred_name = "chexpert_preds.npy" # add name of preds
predictions_dir = predictions_dir / pred_name
np.save(file=predictions_dir, arr=y_pred_avg)

## (Optional) Evaluate Results
If ground truth labels are available, compute AUC on each pathology to evaluate the performance of the zero-shot model. 

In [16]:
y_pred_avg

array([[0.4954721 , 0.4954721 , 0.4954721 , ..., 0.4954721 , 0.4954721 ,
        0.4954721 ],
       [0.47943488, 0.47943488, 0.47943488, ..., 0.47943488, 0.47943488,
        0.47943488],
       [0.49627715, 0.49627715, 0.49627715, ..., 0.49627715, 0.49627715,
        0.49627715],
       ...,
       [0.49723125, 0.49723125, 0.49723125, ..., 0.49723125, 0.49723125,
        0.49723125],
       [0.49545184, 0.49545184, 0.49545184, ..., 0.49545184, 0.49545184,
        0.49545184],
       [0.49585503, 0.49585503, 0.49585503, ..., 0.49585503, 0.49585503,
        0.49585503]], dtype=float32)

In [18]:
# make test_true
test_pred = y_pred_avg
test_true = make_true_labels(cxr_true_labels_path=cxr_true_labels_path, cxr_labels=cxr_labels)

test_pred
# evaluate model
cxr_results = evaluate(test_pred, test_true, cxr_labels)

# boostrap evaluations for 95% confidence intervals
bootstrap_results = bootstrap(test_pred, test_true, cxr_labels)

ValueError: Found input variables with inconsistent numbers of samples: [500, 200]

In [25]:
# display AUC with confidence intervals
bootstrap_results[1]

,Atelectasis_auc,Cardiomegaly_auc,Consolidation_auc,Edema_auc,Enlarged Cardiomediastinum_auc,Fracture_auc,Lung Lesion_auc,Lung Opacity_auc,No Finding_auc,Pleural Effusion_auc,Pleural Other_auc,Pneumonia_auc,Pneumothorax_auc,Support Devices_auc
mean,0.8118,0.9132,0.8901,0.8994,0.9160,0.5603,0.7360,0.9213,0.0700,0.9317,0.6025,0.7798,0.6520,0.7735
lower,0.7720,0.8849,0.8201,0.8662,0.8912,0.2646,0.5658,0.8961,0.0451,0.9053,0.4608,0.5695,0.4854,0.7310
upper,0.8479,0.9367,0.9470,0.9295,0.9375,0.8725,0.8779,0.9426,0.0952,0.9536,0.8855,0.9483,0.8243,0.8130
